## PREDICTING DISPOSITION DESTINATION IN HOSPITALIZED STROKE PATIENTS


In [2]:
import pandas as pd
# load medical charge summaries as DataFrame with pandas
admission_data = pd.read_csv('ADMISSIONS.csv')
# check the loaded data
print('There are %d rows and %d columns in the note events table.'%(admission_data.shape[0], admission_data.shape[1]))
admission_data.head()

There are 58976 rows and 19 columns in the note events table.


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [8]:
# load ICD9 diagnosis code table into pandas dataframe
file_name = 'DIAGNOSES_ICD.csv'
codes_data = pd.read_csv(file_name)
# examine the diagnosis table
print('There are %d rows %d and columns in the diagnosis code table.'\
        %(codes_data.shape[0], codes_data.shape[1]))
codes_data.head()

There are 651047 rows 5 and columns in the diagnosis code table.


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [9]:
codes_data['ICD9_CODE'] = codes_data['ICD9_CODE'].astype('str')
codes_data['is_stroke'] = codes_data.apply(lambda row: 1 if row['ICD9_CODE'][:3]=='434' else 0, axis=1)
codes_data['is_stroke'].value_counts()

0    649567
1      1480
Name: is_stroke, dtype: int64

In [10]:
stroke_codes_data = codes_data[codes_data['is_stroke'] != 0]
stroke_codes_data.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,is_stroke
1474,3010,249,149546,3.0,43411,1
2154,3690,308,166606,8.0,43491,1
2174,3710,313,199765,3.0,43491,1
2280,1784,141,168006,2.0,43491,1
3960,5500,454,182153,1.0,43411,1


Join the Admission and Stroke_code tables

In [11]:
# merge diagnosis code dictionary table to note events/diagnosis code table
stroke_data = pd.merge(admission_data, stroke_codes_data, on = ['SUBJECT_ID', 'HADM_ID'], how = 'inner')

print('There are %d rows and %d columns in the stroke_data table.'\
        %(stroke_data.shape[0], stroke_data.shape[1]))

# examine the joined table
stroke_data.head()

There are 1480 rows and 23 columns in the stroke_data table.


,ROW_ID_x,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,ROW_ID_y,SEQ_NUM,ICD9_CODE,is_stroke
0,190,141,168006,2140-11-06 11:07:00,2140-11-09 20:36:00,2140-11-09 20:36:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,...,OTHER,2140-11-06 07:17:00,2140-11-06 12:07:00,"DRA, SAH",1,1,1784,2.0,43491,1
1,394,308,166606,2135-09-02 23:07:00,2135-10-20 11:53:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Medicaid,...,UNKNOWN/NOT SPECIFIED,2135-09-02 13:18:00,2135-09-03 12:00:00,DILANTIN TOXICITY-MENTAL STATUS CHANGES,0,1,3690,8.0,43491,1
2,398,313,199765,2139-07-26 16:43:00,2139-08-09 15:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,LONG TERM CARE HOSPITAL,Medicare,...,UNKNOWN/NOT SPECIFIED,2139-07-26 15:42:00,2139-07-26 21:08:00,C7 FRACTURE,0,1,3710,3.0,43491,1
3,421,330,197569,2133-02-27 12:30:00,2133-03-24 17:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Private,...,WHITE,NaN,NaN,LEFT CAROTID STENOSIS/SDA,0,1,3895,6.0,43411,1
4,445,352,121050,2144-05-17 15:47:00,2144-05-21 11:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,WHITE,2144-05-17 10:54:00,2144-05-17 16:03:00,STROKE;TELEMETRY,0,1,4101,1.0,43411,1


### read in the patient information table

In [12]:
# load PATIENTS.csv file
patient_information = pd.read_csv('PATIENTS.csv')
patient_information.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


### join stroke_data table and patient_information table

In [13]:
stroke_data = pd.merge(stroke_data, patient_information, on = ['SUBJECT_ID'], how = 'inner')
print('There are %d rows and %d columns in the stroke_data table.'\
        %(stroke_data.shape[0], stroke_data.shape[1]))

# examine the joined table
stroke_data.head()


There are 1480 rows and 30 columns in the stroke_data table.


,ROW_ID_x,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,SEQ_NUM,ICD9_CODE,is_stroke,ROW_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,190,141,168006,2140-11-06 11:07:00,2140-11-09 20:36:00,2140-11-09 20:36:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,...,2.0,43491,1,131,M,2060-05-03 00:00:00,2140-11-09 00:00:00,2140-11-09 00:00:00,2140-11-09 00:00:00,1
1,394,308,166606,2135-09-02 23:07:00,2135-10-20 11:53:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Medicaid,...,8.0,43491,1,289,F,2106-03-01 00:00:00,2135-10-28 00:00:00,NaN,2135-10-28 00:00:00,1
2,398,313,199765,2139-07-26 16:43:00,2139-08-09 15:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,LONG TERM CARE HOSPITAL,Medicare,...,3.0,43491,1,293,F,2060-11-20 00:00:00,2142-09-11 00:00:00,NaN,2142-09-11 00:00:00,1
3,421,330,197569,2133-02-27 12:30:00,2133-03-24 17:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Private,...,6.0,43411,1,310,M,2065-06-10 00:00:00,2139-01-06 00:00:00,NaN,2139-01-06 00:00:00,1
4,445,352,121050,2144-05-17 15:47:00,2144-05-21 11:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,1.0,43411,1,331,M,2069-03-06 00:00:00,2149-05-15 00:00:00,NaN,2149-05-15 00:00:00,1


### Create a new column "age" (ADMITTIME - DOB) in table stroke_data 

In [14]:
# convert ADMITTIME AND DOB columns to datetime type
stroke_data["ADMITTIME"] = pd.to_datetime(stroke_data["ADMITTIME"])
stroke_data["DOB"] = pd.to_datetime(stroke_data["DOB"])

# add a new column "age" by substracting DOB from ADMITTIME
stroke_data["Age"] = stroke_data["ADMITTIME"].dt.year - stroke_data["DOB"].dt.year
stroke_data.head()

,ROW_ID_x,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,ICD9_CODE,is_stroke,ROW_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,Age
0,190,141,168006,2140-11-06 11:07:00,2140-11-09 20:36:00,2140-11-09 20:36:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,...,43491,1,131,M,2060-05-03,2140-11-09 00:00:00,2140-11-09 00:00:00,2140-11-09 00:00:00,1,80
1,394,308,166606,2135-09-02 23:07:00,2135-10-20 11:53:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Medicaid,...,43491,1,289,F,2106-03-01,2135-10-28 00:00:00,NaN,2135-10-28 00:00:00,1,29
2,398,313,199765,2139-07-26 16:43:00,2139-08-09 15:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,LONG TERM CARE HOSPITAL,Medicare,...,43491,1,293,F,2060-11-20,2142-09-11 00:00:00,NaN,2142-09-11 00:00:00,1,79
3,421,330,197569,2133-02-27 12:30:00,2133-03-24 17:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Private,...,43411,1,310,M,2065-06-10,2139-01-06 00:00:00,NaN,2139-01-06 00:00:00,1,68
4,445,352,121050,2144-05-17 15:47:00,2144-05-21 11:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,43411,1,331,M,2069-03-06,2149-05-15 00:00:00,NaN,2149-05-15 00:00:00,1,75


### Drop unnecessary columns

In [15]:
stroke_data = stroke_data[["SUBJECT_ID", "HADM_ID", "ADMITTIME", "ADMISSION_TYPE", "ADMISSION_LOCATION", "INSURANCE", "GENDER", "Age", "ICD9_CODE", "DISCHARGE_LOCATION"]]
stroke_data.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,ADMISSION_TYPE,ADMISSION_LOCATION,INSURANCE,GENDER,Age,ICD9_CODE,DISCHARGE_LOCATION
0,141,168006,2140-11-06 11:07:00,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,M,80,43491,DEAD/EXPIRED
1,308,166606,2135-09-02 23:07:00,EMERGENCY,EMERGENCY ROOM ADMIT,Medicaid,F,29,43491,DISC-TRAN CANCER/CHLDRN H
2,313,199765,2139-07-26 16:43:00,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,F,79,43491,LONG TERM CARE HOSPITAL
3,330,197569,2133-02-27 12:30:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,Private,M,68,43411,SNF
4,352,121050,2144-05-17 15:47:00,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,M,75,43411,SNF


### Read NOTEEVENTS.csv

In [16]:
note_event = pd.read_csv("NOTEEVENTS.csv")
note_event.head()

/Applications/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


### Read CAREGIVERS.csv

In [17]:
caregiver = pd.read_csv("CAREGIVERS.csv")
caregiver.head()

,ROW_ID,CGID,LABEL,DESCRIPTION
0,2228,16174,RO,Read Only
1,2229,16175,RO,Read Only
2,2230,16176,Res,Resident/Fellow/PA/NP
3,2231,16177,RO,Read Only
4,2232,16178,RT,Respiratory


### Join note_event and caregiver tables to identify notes' type

In [18]:
note_data = pd.merge(note_event, caregiver, on = ['CGID'], how = 'inner')

#keep the columns that we are interested in
note_data = note_data[["SUBJECT_ID", "HADM_ID", "LABEL", "TEXT"]]
note_data.head()

,SUBJECT_ID,HADM_ID,LABEL,TEXT
0,29075,179159.0,RN,"67M w/ h/o multiplemyeloma Dx [**2111**], neur..."
1,5691,148740.0,RN,Ms. [**Known lastname 1234**] is a 77 yo femal...
2,12113,197423.0,RN,NPN\n Neuro: Pt was oriented to name only ...
3,31608,152365.0,RN,"Respiratory failure, acute (not ARDS/[**Doctor..."
4,29075,179159.0,RN,.H/O back pain\n Assessment:\n Action:\n ...


### Add the note feature to stroke_data table

In [19]:
stroke_data = pd.merge(stroke_data, note_data, on = ['SUBJECT_ID', 'HADM_ID'], how = 'inner')
stroke_data.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,ADMISSION_TYPE,ADMISSION_LOCATION,INSURANCE,GENDER,Age,ICD9_CODE,DISCHARGE_LOCATION,LABEL,TEXT
0,141,168006,2140-11-06 11:07:00,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,M,80,43491,DEAD/EXPIRED,RN,MICU NPN Update Code status:\nFamily met with ...
1,141,168006,2140-11-06 11:07:00,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,M,80,43491,DEAD/EXPIRED,RN,MICU NPN Update:\nPt was extubated just a litt...
2,141,168006,2140-11-06 11:07:00,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,M,80,43491,DEAD/EXPIRED,RRT,"Patient remains on mechanical ventilation, tra..."
3,141,168006,2140-11-06 11:07:00,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,M,80,43491,DEAD/EXPIRED,RN,Admit note 80 yo black male NH resident found ...
4,141,168006,2140-11-06 11:07:00,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,M,80,43491,DEAD/EXPIRED,RN,"Add 1830p given 2 uffp inr 1.6, bc x2 sent, na..."


### Read LABEVENTS.csv

In [5]:
lab_event = pd.read_csv("LABEVENTS.csv")

print('There are %d rows and %d columns in the lab_event table.'\
        %(lab_event.shape[0], lab_event.shape[1]))
lab_event.head()


There are 27854055 rows and 9 columns in the lab_event table.


,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal


### Read D_LABITEMS.csv


In [3]:
lab_id = pd.read_csv("D_LABITEMS.csv")

print('There are %d rows and %d columns in the lab_event table.'\
        %(lab_id.shape[0], lab_id.shape[1]))
lab_id.head()

There are 753 rows and 6 columns in the lab_event table.


,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
0,546,51346,Blasts,Cerebrospinal Fluid (CSF),Hematology,26447-3
1,547,51347,Eosinophils,Cerebrospinal Fluid (CSF),Hematology,26451-5
2,548,51348,"Hematocrit, CSF",Cerebrospinal Fluid (CSF),Hematology,30398-2
3,549,51349,Hypersegmented Neutrophils,Cerebrospinal Fluid (CSF),Hematology,26506-6
4,550,51350,Immunophenotyping,Cerebrospinal Fluid (CSF),Hematology,NaN


### join lab_event and lab_id on ITEMID

In [6]:
lab_event = pd.merge(lab_event, lab_id, on = "ITEMID", how = "inner").drop_duplicates()
print('There are %d rows and %d columns in the lab_event table.'\
        %(lab_event.shape[0], lab_event.shape[1]))
lab_event.head()

There are 27854055 rows and 14 columns in the lab_event table.


,ROW_ID_x,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG,ROW_ID_y,LABEL,FLUID,CATEGORY,LOINC_CODE
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN,21,pH,Blood,Blood Gas,11558-4
1,289,3,NaN,50820,2101-10-12 18:17:00,7.42,7.42,units,NaN,21,pH,Blood,Blood Gas,11558-4
2,396,3,NaN,50820,2101-10-15 12:42:00,7.49,7.49,units,NaN,21,pH,Blood,Blood Gas,11558-4
3,976,3,145834.0,50820,2101-10-22 04:31:00,7.40,7.40,units,NaN,21,pH,Blood,Blood Gas,11558-4
4,989,3,145834.0,50820,2101-10-22 07:13:00,7.37,7.37,units,NaN,21,pH,Blood,Blood Gas,11558-4


### filter the lab_event table to include the stroke patients' lab events only

In [20]:
stroke_id = stroke_data[["SUBJECT_ID", "ADMITTIME"]]
stroke_lab_event = pd.merge(lab_event, stroke_id, on = "SUBJECT_ID", how = "inner").drop_duplicates()
print('There are %d rows and %d columns in the stroke_lab_event table.'\
        %(stroke_lab_event.shape[0], stroke_lab_event.shape[1]))
stroke_lab_event.head()

There are 937082 rows and 15 columns in the stroke_lab_event table.


,ROW_ID_x,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG,ROW_ID_y,LABEL,FLUID,CATEGORY,LOINC_CODE,ADMITTIME
0,78717,141,168006.0,50820,2140-11-06 12:47:00,7.27,7.27,units,abnormal,21,pH,Blood,Blood Gas,11558-4,2140-11-06 11:07:00
13,78725,141,168006.0,50820,2140-11-06 14:43:00,7.26,7.26,units,abnormal,21,pH,Blood,Blood Gas,11558-4,2140-11-06 11:07:00
26,78731,141,168006.0,50820,2140-11-06 16:17:00,7.30,7.30,units,abnormal,21,pH,Blood,Blood Gas,11558-4,2140-11-06 11:07:00
39,78754,141,168006.0,50820,2140-11-06 18:40:00,7.36,7.36,units,NaN,21,pH,Blood,Blood Gas,11558-4,2140-11-06 11:07:00
52,78791,141,168006.0,50820,2140-11-06 20:43:00,7.36,7.36,units,NaN,21,pH,Blood,Blood Gas,11558-4,2140-11-06 11:07:00


### Filter the lab events that happen within 24 hours of admission

In [21]:
# convert ADMITTIME AND CHARTTIME columns to datetime type
stroke_lab_event["ADMITTIME"] = pd.to_datetime(stroke_lab_event["ADMITTIME"])
stroke_lab_event["CHARTTIME"] = pd.to_datetime(stroke_lab_event["CHARTTIME"])

# add a new column "TimeDiff"
stroke_lab_event["TimeDiff"] = abs((stroke_lab_event["ADMITTIME"] - stroke_lab_event["CHARTTIME"]).astype('timedelta64[h]'))
stroke_lab_event.head()

#filter the lab events within 24 hours of admission
filtered_stroke_lab_event = stroke_lab_event[stroke_lab_event["TimeDiff"] <= 24]
print('There are %d rows and %d columns in the filtered_stroke_lab_event table.'\
        %(filtered_stroke_lab_event.shape[0], filtered_stroke_lab_event.shape[1]))
filtered_stroke_lab_event.head()

There are 117901 rows and 16 columns in the filtered_stroke_lab_event table.


,ROW_ID_x,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG,ROW_ID_y,LABEL,FLUID,CATEGORY,LOINC_CODE,ADMITTIME,TimeDiff
0,78717,141,168006.0,50820,2140-11-06 12:47:00,7.27,7.27,units,abnormal,21,pH,Blood,Blood Gas,11558-4,2140-11-06 11:07:00,2.0
13,78725,141,168006.0,50820,2140-11-06 14:43:00,7.26,7.26,units,abnormal,21,pH,Blood,Blood Gas,11558-4,2140-11-06 11:07:00,4.0
26,78731,141,168006.0,50820,2140-11-06 16:17:00,7.30,7.30,units,abnormal,21,pH,Blood,Blood Gas,11558-4,2140-11-06 11:07:00,6.0
39,78754,141,168006.0,50820,2140-11-06 18:40:00,7.36,7.36,units,NaN,21,pH,Blood,Blood Gas,11558-4,2140-11-06 11:07:00,8.0
52,78791,141,168006.0,50820,2140-11-06 20:43:00,7.36,7.36,units,NaN,21,pH,Blood,Blood Gas,11558-4,2140-11-06 11:07:00,10.0


### Filter out the rows that are not lab tests related to stroke

In [22]:
filtered_stroke_lab_event = filtered_stroke_lab_event.loc[filtered_stroke_lab_event["LOINC_CODE"].isin(['26454-9','26465-5','2160-0','2823-3','3094-0','2345-7','33914-3','5902-2','742-7','711-2','6768-6','8122-4','2039-6','1751-7','4544-3','2085-9','2951-2','2093-3','2500-7','2498-4','2075-0'])]
print('There are %d rows and %d columns in the filtered_stroke_lab_event table.'\
        %(filtered_stroke_lab_event.shape[0], filtered_stroke_lab_event.shape[1]))
filtered_stroke_lab_event.head()

There are 23981 rows and 16 columns in the filtered_stroke_lab_event table.


,ROW_ID_x,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG,ROW_ID_y,LABEL,FLUID,CATEGORY,LOINC_CODE,ADMITTIME,TimeDiff
1547,78685,141,168006.0,50902,2140-11-06 12:00:00,105,105.0,mEq/L,NaN,103,Chloride,Blood,Chemistry,2075-0,2140-11-06 11:07:00,1.0
1560,78736,141,168006.0,50902,2140-11-06 16:18:00,108,108.0,mEq/L,NaN,103,Chloride,Blood,Chemistry,2075-0,2140-11-06 11:07:00,6.0
1573,78759,141,168006.0,50902,2140-11-06 20:30:00,109,109.0,mEq/L,abnormal,103,Chloride,Blood,Chemistry,2075-0,2140-11-06 11:07:00,10.0
1586,78841,141,168006.0,50902,2140-11-07 04:11:00,106,106.0,mEq/L,NaN,103,Chloride,Blood,Chemistry,2075-0,2140-11-06 11:07:00,18.0
1599,78866,141,168006.0,50902,2140-11-07 05:00:00,107,107.0,mEq/L,NaN,103,Chloride,Blood,Chemistry,2075-0,2140-11-06 11:07:00,18.0


### Filter out the rows with VALUENUM to be NULL

In [23]:
filtered_stroke_lab_event = filtered_stroke_lab_event[["SUBJECT_ID", "ITEMID", "VALUENUM", "TimeDiff"]]
filtered_stroke_lab_event = filtered_stroke_lab_event.dropna()
print('There are %d rows and %d columns in the filtered_stroke_lab_event table.'\
        %(filtered_stroke_lab_event.shape[0], filtered_stroke_lab_event.shape[1]))
filtered_stroke_lab_event.head()

There are 23557 rows and 4 columns in the filtered_stroke_lab_event table.


,SUBJECT_ID,ITEMID,VALUENUM,TimeDiff
1547,141,50902,105.0,1.0
1560,141,50902,108.0,6.0
1573,141,50902,109.0,10.0
1586,141,50902,106.0,18.0
1599,141,50902,107.0,18.0


### Aggregate the filtered_stroke_lab_event table by SUBJECT_ID
#### Create a new column to combine the lab ITEMID and hour after admission, "LAB_HOUR", which is served as feature_id
#### Create a dictionary for patient_lab_features while contains an array of tuples (feature_id, feature_value)


In [24]:

# create a new column "LAB_HOUR"
filtered_stroke_lab_event["LAB_HOUR"] = filtered_stroke_lab_event["ITEMID"].astype("str") + "_" + filtered_stroke_lab_event["TimeDiff"].astype("str")

# map the created feature_id "LAB_HOUR" to index
lab_feature_map = pd.DataFrame(filtered_stroke_lab_event["LAB_HOUR"].unique())
lab_feature_map["feature_idx"] = lab_feature_map.index + 1
lab_feature_map.columns = ["LAB_HOUR", "feature_idx"]

#join filtered_stroke_lab_event and lab_feature_map
filtered_stroke_lab_event = pd.merge(filtered_stroke_lab_event, lab_feature_map, on = "LAB_HOUR", how = "inner")

#if multiple same lab events happen at the same time, take the mean value
filtered_stroke_lab_event = filtered_stroke_lab_event.groupby(["SUBJECT_ID", "feature_idx"])["VALUENUM"].agg("mean").reset_index()

# create a new column "LABID_VALUE" which is a tuple (feature_idx, VALUE)
filtered_stroke_lab_event["LABID_VALUE"] = filtered_stroke_lab_event[["feature_idx", "VALUENUM"]].apply(tuple, axis=1)

# create a dictionary of patient_lab_features
new_filtered_stroke_lab_event = filtered_stroke_lab_event[["SUBJECT_ID", "LABID_VALUE"]]
patient_lab_features = new_filtered_stroke_lab_event.groupby("SUBJECT_ID")["LABID_VALUE"].apply(lambda x: [x for x in x.values]).to_dict()


### Convert patient_lab_features to svmlight format

In [25]:
lab_feature = open("./patient_lab_features_svmlight.data", 'w')
data = []

for key, value in sorted(patient_lab_features.items()):
    each_data = []
    each_data.append(str(int(key)))
    
    for (x1, x2) in sorted(value):
        each_data.append("%s:%.4f" % (int(x1), x2))
        
    each_data_str = ' '.join(each_data)
        
    data.append(each_data_str)
            
lab_feature.write("\n".join(data))
lab_feature.close()


### Reload patient_lab_features_svmlight.data into sparse matrix

In [26]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_svmlight_file

patient_lab_features_sparse = load_svmlight_file("./patient_lab_features_svmlight.data")

patient_lab_features_sparse_value = patient_lab_features_sparse[0]
patient_lab_features_sparse_id = patient_lab_features_sparse[1]

patient_lab_features_sparse_dataframe = pd.SparseDataFrame([ pd.SparseSeries(patient_lab_features_sparse_value[i].toarray().ravel()) 
                              for i in np.arange(patient_lab_features_sparse_value.shape[0]) ])


patient_lab_features_sparse_dataframe["SUBJECT_ID"] = patient_lab_features_sparse_id

patient_lab_features_sparse_dataframe.head()


,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,SUBJECT_ID
0,105.0,108.0,109.0,106.5,97.0,1.6,1.6,1.5,1.4,1.35,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141.0
1,105.0,0.0,108.0,0.0,0.0,0.0,1.4,0.0,1.3,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,249.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,308.0
3,104.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,313.0
4,105.0,0.0,0.0,0.0,108.0,1.2,1.2,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,330.0


### Read MICROBIOLOGYEVENTS.csv

In [192]:
bio_event = pd.read_csv("MICROBIOLOGYEVENTS.csv")

print('There are %d rows and %d columns in the bio_event table.'\
        %(bio_event.shape[0], bio_event.shape[1]))
bio_event.head()


There are 631726 rows and 16 columns in the bio_event table.


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION
0,744,96,170324,2156-04-13 00:00:00,2156-04-13 14:18:00,70021.0,BRONCHOALVEOLAR LAVAGE,80026.0,PSEUDOMONAS AERUGINOSA,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,745,96,170324,2156-04-20 00:00:00,2156-04-20 13:10:00,70062.0,SPUTUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,746,96,170324,2156-04-20 00:00:00,2156-04-20 16:00:00,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,747,96,170324,2156-04-20 00:00:00,NaN,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,748,96,170324,2156-04-20 00:00:00,NaN,70079.0,URINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Filter the bio_event table to include the stroke patients' microbiology events only

In [193]:
stroke_bio_event = pd.merge(bio_event, stroke_id, on = "SUBJECT_ID", how = "inner").drop_duplicates()
print('There are %d rows and %d columns in the stroke_bio_event table.'\
        %(stroke_bio_event.shape[0], stroke_bio_event.shape[1]))
stroke_bio_event.head()

There are 25986 rows and 17 columns in the stroke_bio_event table.


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION,ADMITTIME
0,1301,141,168006,2140-11-06 00:00:00,2140-11-06 08:10:00,70079.0,URINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2140-11-06 11:07:00
13,1302,141,168006,2140-11-06 00:00:00,2140-11-06 12:33:00,70069.0,SWAB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2140-11-06 11:07:00
26,1303,141,168006,2140-11-06 00:00:00,2140-11-06 13:00:00,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2140-11-06 11:07:00
39,1304,141,168006,2140-11-06 00:00:00,2140-11-06 17:20:00,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2140-11-06 11:07:00
52,1305,141,168006,2140-11-08 00:00:00,2140-11-08 18:00:00,70062.0,SPUTUM,80018.0,MORGANELLA MORGANII,1.0,90017.0,CEFTAZIDIME,<=1,<=,1.0,S,2140-11-06 11:07:00


### Filter the microbiology events that happen within 24 hours of admission

In [201]:
# convert ADMITTIME AND CHARTTIME columns to datetime type
stroke_bio_event["ADMITTIME"] = pd.to_datetime(stroke_bio_event["ADMITTIME"])
stroke_bio_event["CHARTTIME"] = pd.to_datetime(stroke_bio_event["CHARTTIME"])

# add a new column "TimeDiff"
stroke_bio_event["TimeDiff"] = abs((stroke_bio_event["ADMITTIME"] - stroke_bio_event["CHARTTIME"]).astype('timedelta64[h]'))

#filter the lab events within 24 hours of admission
filtered_stroke_bio_event = stroke_bio_event[stroke_bio_event["TimeDiff"] <= 24]
print('There are %d rows and %d columns in the filtered_stroke_bio_event table.'\
        %(filtered_stroke_bio_event.shape[0], filtered_stroke_bio_event.shape[1]))
filtered_stroke_bio_event.head()

There are 3028 rows and 18 columns in the filtered_stroke_bio_event table.


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION,ADMITTIME,TimeDiff
0,1301,141,168006,2140-11-06 00:00:00,2140-11-06 08:10:00,70079.0,URINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2140-11-06 11:07:00,2.0
13,1302,141,168006,2140-11-06 00:00:00,2140-11-06 12:33:00,70069.0,SWAB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2140-11-06 11:07:00,2.0
26,1303,141,168006,2140-11-06 00:00:00,2140-11-06 13:00:00,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2140-11-06 11:07:00,2.0
39,1304,141,168006,2140-11-06 00:00:00,2140-11-06 17:20:00,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2140-11-06 11:07:00,7.0
906,2715,249,149546,2155-02-03 00:00:00,2155-02-03 23:20:00,70091.0,MRSA SCREEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2155-02-03 20:16:00,4.0


In [202]:
# create a new column "BIO_HOUR"
filtered_stroke_bio_event["BIO_HOUR"] = filtered_stroke_bio_event["SPEC_ITEMID"].astype("str") + "_" + filtered_stroke_med_event["TimeDiff"].astype("str")

# map the created feature_id "BIO_HOUR" to index
bio_feature_map = pd.DataFrame(filtered_stroke_bio_event["BIO_HOUR"].unique())
bio_feature_map["feature_idx"] = bio_feature_map.index + 1
bio_feature_map.columns = ["BIO_HOUR", "feature_idx"]

#join filtered_stroke_bio_event and bio_feature_map
filtered_stroke_bio_event = pd.merge(filtered_stroke_bio_event, bio_feature_map, on = "BIO_HOUR", how = "inner")
filtered_stroke_bio_event["VALUE"] = 1

# create a new column "BIOID_VALUE" which is a tuple (feature_idx, VALUE)
filtered_stroke_bio_event["BIOID_VALUE"] = filtered_stroke_bio_event[["feature_idx", "VALUE"]].apply(tuple, axis=1)

# create a dictionary of patient_med_features
new_filtered_stroke_bio_event = filtered_stroke_bio_event[["SUBJECT_ID", "BIOID_VALUE"]].drop_duplicates()
patient_bio_features = new_filtered_stroke_bio_event.groupby("SUBJECT_ID")["BIOID_VALUE"].apply(lambda x: [x for x in x.values]).to_dict()


/Applications/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [204]:
bio_feature = open("./patient_bio_features_svmlight.data", 'w')
data = []

for key, value in sorted(patient_bio_features.items()):
    each_data = []
    each_data.append(str(int(key)))
    
    for (x1, x2) in sorted(value):
        each_data.append("%s:%s" % (int(x1), int(x2)))
        
    each_data_str = ' '.join(each_data)
        
    data.append(each_data_str)
            
bio_feature.write("\n".join(data))
bio_feature.close()


In [205]:
patient_bio_features_sparse = load_svmlight_file("./patient_bio_features_svmlight.data")

patient_bio_features_sparse_value = patient_bio_features_sparse[0]
patient_bio_features_sparse_id = patient_bio_features_sparse[1]

patient_bio_features_sparse_dataframe = pd.SparseDataFrame([ pd.SparseSeries(patient_bio_features_sparse_value[i].toarray().ravel()) 
                              for i in np.arange(patient_bio_features_sparse_value.shape[0]) ])


patient_bio_features_sparse_dataframe["SUBJECT_ID"] = patient_bio_features_sparse_id

patient_bio_features_sparse_dataframe.head()


,0,1,2,3,4,5,6,7,8,9,...,231,232,233,234,235,236,237,238,239,SUBJECT_ID
0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,249.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,308.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,313.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,624.0


### Read PRESCRIPTIONS.csv

In [206]:
med_event = pd.read_csv("PRESCRIPTIONS.csv")

print('There are %d rows and %d columns in the med_event table.'\
        %(med_event.shape[0], med_event.shape[1]))
med_event.head()


/Applications/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are 4156450 rows and 19 columns in the med_event table.


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20,mg,1,TAB,PO


In [207]:
stroke_med_event = pd.merge(med_event, stroke_id, on = "SUBJECT_ID", how = "inner").drop_duplicates()
print('There are %d rows and %d columns in the stroke_med_event table.'\
        %(stroke_med_event.shape[0], stroke_med_event.shape[1]))
stroke_med_event.head()

There are 134710 rows and 20 columns in the stroke_med_event table.


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,ADMITTIME
0,1656787,141,168006,234668.0,2140-11-06 00:00:00,2140-11-07 00:00:00,MAIN,Quetiapine Fumarate,Quetiapine Fumarate,Quetiapine Fumarate,QUET100,034188,310027139.0,100mg Tab,100,mg,1,TAB,PO,2140-11-06 11:07:00
13,1657446,141,168006,234668.0,2140-11-06 00:00:00,2140-11-07 00:00:00,MAIN,Magnesium Sulfate,NaN,NaN,MAGS1I,001412,74407532.0,1gm/2ml vial,3,gm,6,ml,IV,2140-11-06 11:07:00
26,1657445,141,168006,234668.0,2140-11-06 00:00:00,2140-11-07 00:00:00,MAIN,Famotidine,NaN,NaN,FAMO20PM,021732,6353750.0,20mg Premix Bag,20,mg,1,BAG,IV,2140-11-06 11:07:00
39,1656794,141,168006,234668.0,2140-11-06 00:00:00,2140-11-07 00:00:00,MAIN,Morphine Sulfate,Morphine Sulfate,Morphine Sulfate,MORP2I,004070,74176201.0,2mg Syringe,2,mg,1,SYR,IV,2140-11-06 11:07:00
52,1656797,141,168006,234668.0,2140-11-06 00:00:00,2140-11-07 00:00:00,MAIN,Metoprolol,Metoprolol,Metoprolol,METO5I,025856,74177825.0,5MG/5ML VIAL,5,mg,1,VIAL,IV,2140-11-06 11:07:00


In [213]:
# convert ADMITTIME AND CHARTTIME columns to datetime type
stroke_med_event["ADMITTIME"] = pd.to_datetime(stroke_med_event["ADMITTIME"])
stroke_med_event["STARTDATE"] = pd.to_datetime(stroke_med_event["STARTDATE"])

# add a new column "TimeDiff"
stroke_med_event["TimeDiff"] = abs((stroke_med_event["ADMITTIME"] - stroke_med_event["STARTDATE"]).astype('timedelta64[h]'))

#filter the lab events within 24 hours of admission
filtered_stroke_med_event = stroke_med_event[stroke_med_event["TimeDiff"] <= 24]
print('There are %d rows and %d columns in the filtered_stroke_med_event table.'\
        %(filtered_stroke_med_event.shape[0], filtered_stroke_med_event.shape[1]))
filtered_stroke_med_event.head()

There are 29166 rows and 21 columns in the filtered_stroke_med_event table.


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,...,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,ADMITTIME,TimeDiff
0,1656787,141,168006,234668.0,2140-11-06,2140-11-07 00:00:00,MAIN,Quetiapine Fumarate,Quetiapine Fumarate,Quetiapine Fumarate,...,034188,310027139.0,100mg Tab,100,mg,1,TAB,PO,2140-11-06 11:07:00,11.0
13,1657446,141,168006,234668.0,2140-11-06,2140-11-07 00:00:00,MAIN,Magnesium Sulfate,NaN,NaN,...,001412,74407532.0,1gm/2ml vial,3,gm,6,ml,IV,2140-11-06 11:07:00,11.0
26,1657445,141,168006,234668.0,2140-11-06,2140-11-07 00:00:00,MAIN,Famotidine,NaN,NaN,...,021732,6353750.0,20mg Premix Bag,20,mg,1,BAG,IV,2140-11-06 11:07:00,11.0
39,1656794,141,168006,234668.0,2140-11-06,2140-11-07 00:00:00,MAIN,Morphine Sulfate,Morphine Sulfate,Morphine Sulfate,...,004070,74176201.0,2mg Syringe,2,mg,1,SYR,IV,2140-11-06 11:07:00,11.0
52,1656797,141,168006,234668.0,2140-11-06,2140-11-07 00:00:00,MAIN,Metoprolol,Metoprolol,Metoprolol,...,025856,74177825.0,5MG/5ML VIAL,5,mg,1,VIAL,IV,2140-11-06 11:07:00,11.0


In [214]:
# create a new column "MED_HOUR"
filtered_stroke_med_event["MED_HOUR"] = filtered_stroke_med_event["DRUG"].astype("str") + "_" + filtered_stroke_med_event["TimeDiff"].astype("str")

# map the created feature_id "MED_HOUR" to index
med_feature_map = pd.DataFrame(filtered_stroke_med_event["MED_HOUR"].unique())
med_feature_map["feature_idx"] = med_feature_map.index + 1
med_feature_map.columns = ["MED_HOUR", "feature_idx"]

#join filtered_stroke_med_event and med_feature_map
filtered_stroke_med_event = pd.merge(filtered_stroke_med_event, med_feature_map, on = "MED_HOUR", how = "inner")
filtered_stroke_med_event["VALUE"] = 1

# create a new column "BIOID_VALUE" which is a tuple (feature_idx, VALUE)
filtered_stroke_med_event["MEDID_VALUE"] = filtered_stroke_med_event[["feature_idx", "VALUE"]].apply(tuple, axis=1)

# create a dictionary of patient_med_features
new_filtered_stroke_med_event = filtered_stroke_med_event[["SUBJECT_ID", "MEDID_VALUE"]].drop_duplicates()
patient_med_features = new_filtered_stroke_med_event.groupby("SUBJECT_ID")["MEDID_VALUE"].apply(lambda x: [x for x in x.values]).to_dict()


/Applications/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [215]:
med_feature = open("./patient_med_features_svmlight.data", 'w')
data = []

for key, value in sorted(patient_med_features.items()):
    each_data = []
    each_data.append(str(int(key)))
    
    for (x1, x2) in sorted(value):
        each_data.append("%s:%s" % (int(x1), int(x2)))
        
    each_data_str = ' '.join(each_data)
        
    data.append(each_data_str)
            
med_feature.write("\n".join(data))
med_feature.close()


In [216]:
patient_med_features_sparse = load_svmlight_file("./patient_med_features_svmlight.data")

patient_med_features_sparse_value = patient_med_features_sparse[0]
patient_med_features_sparse_id = patient_med_features_sparse[1]

patient_med_features_sparse_dataframe = pd.SparseDataFrame([ pd.SparseSeries(patient_med_features_sparse_value[i].toarray().ravel()) 
                              for i in np.arange(patient_med_features_sparse_value.shape[0]) ])


patient_med_features_sparse_dataframe["SUBJECT_ID"] = patient_med_features_sparse_id

patient_med_features_sparse_dataframe.head()


,0,1,2,3,4,5,6,7,8,9,...,5103,5104,5105,5106,5107,5108,5109,5110,5111,SUBJECT_ID
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,249.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,313.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,330.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,352.0
